# Game of the Year (GOTY) Data Analysis

This project focuses on analyzing Game of the Year (GOTY) nominees and winners from 2014 to 2024.  
The original goal was to build a Machine Learning model to predict future winners, but due to the limited amount of available data, the project was finalized as a **data analysis and presentation** instead.

The workflow includes:
- Collecting GOTY nominees and winners.
- Scraping data from Metacritic using Selenium and BeautifulSoup.
- Cleaning and structuring the dataset into a final table with relevant features (critic score, user score, sentiment from reviews, genres, platforms, developers, publishers, etc.).
- Preparing the results for visualization and presentation.

The Jupyter Notebook documents the entire pipeline (extraction → cleaning → organization → final dataset).  
The final presentation is hosted on a website, with this Notebook linked as a technical reference.


#### About this Notebook (Purpose & Scope)

This notebook is **not** a presentation.  
It serves as a **technical demonstration** of the data-extraction pipeline used in the GOTY project, for anyone who wants to inspect or reproduce the process.

**What you’ll find here**
- How the GOTY nominees/winners list (2014–2024) is defined and turned into Metacritic URLs.
- Selenium + BeautifulSoup routines to scrape titles, scores, release dates, platforms, developers, publishers, and user-review links.
- A robust review harvester (infinite-scroll handling + year filter) and **sentiment tagging** with a pretrained Transformer.
- Basic string cleaning (`.strip()`, `.replace()`, `.split()`) and column reordering.
- Exports of the final tables (e.g., `GOTY_YEARS.csv/.xlsx`, review expansions).

**What this is not**
- Not a polished report or storytelling artifact (the **final presentation lives on the website** and links back to this notebook).
- Not a prediction/ML project (ML was intentionally skipped due to limited data volume).
- Not a general scraping tutorial; the code is tailored to Metacritic’s structure and may require selector updates if the site changes.

**Reproducibility & Etiquette**
- Requires a local Selenium/ChromeDriver setup and internet access.
- Includes waits to avoid overloading the site; please respect Metacritic’s ToS and run responsibly.


#### Libraries:

In [2]:
from selenium import webdriver   # Main tool for web scraping and navigation
import time                      # Used to add delays and avoid overloading the server
from bs4 import BeautifulSoup    # Library for parsing and extracting structured data from HTML
import pandas as pd              # Library for data manipulation and analysis


### List of Games


At this stage, the goal is **data extraction**.  
We define the dictionary `goty_all` as a structured list of all Game of the Year (GOTY) nominees and winners from 2014 to 2024.  

- **Keys** → represent each year.  
- **Values** → contain the games nominated in that year.  
- The GOTY winner for each year is marked with a comment (`# Winner`).  

This dictionary is later used to **build the URLs** for each game on Metacritic, serving as the starting point for the web scraping process.


In [4]:
goty_all = {
    2014: [
        "dragon-age-inquisition",  # Winner
        "bayonetta-2",
        "dark-souls-ii",
        "hearthstone-heroes-of-warcraft",
        "middle-earth-shadow-of-mordor"
    ],
    2015: [
        "the-witcher-3-wild-hunt",  # Winner
        "bloodborne",
        "fallout-4",
        "metal-gear-solid-v-the-phantom-pain",
        "super-mario-maker"
    ], 
    2016: [
        "overwatch",  # Winner
        "doom",
        "inside",
        "titanfall-2",
        "uncharted-4-a-thiefs-end"
    ],
    2017: [
        "the-legend-of-zelda-breath-of-the-wild",  # Winner
        "horizon-zero-dawn",
        "persona-5",
        "playerunknowns-battlegrounds",
        "super-mario-odyssey"
    ],
    2018: [
        "god-of-war",  # Winner
        "assassins-creed-odyssey",
        "celeste",
        "marvels-spider-man",
        "monster-hunter-world",
        "red-dead-redemption-2"
    ],
    2019: [
        "sekiro-shadows-die-twice",  # Winner
        "control",
        "death-stranding",
        "resident-evil-2",
        "super-smash-bros-ultimate",
        "the-outer-worlds"
    ],
    2020: [
        "the-last-of-us-part-ii",  # Winner
        "animal-crossing-new-horizons",
        "doom-eternal",
        "final-fantasy-vii-remake",
        "ghost-of-tsushima",
        "hades"
    ],
    2021: [
        "it-takes-two",  # Winner
        "deathloop",
        "metroid-dread",
        "psychonauts-2",
        "ratchet-and-clank-rift-apart",
        "resident-evil-village"
    ],
    2022: [
        "elden-ring",  # Winner
        "a-plague-tale-requiem",
        "god-of-war-ragnarok",
        "horizon-forbidden-west",
        "stray",
        "xenoblade-chronicles-3"
    ],
    2023: [
        "baldurs-gate-3",  # Winner
        "alan-wake-ii",
        "the-legend-of-zelda-tears-of-the-kingdom",
        "marvels-spider-man-2",
        "resident-evil-4",
        "super-mario-bros-wonder"
    ],
    2024: [
        "astro-bot",  # Winner
        "balatro",
        "black-myth-wukong",
        "elden-ring-shadow-of-the-erdtree",
        "final-fantasy-vii-rebirth",
        "metaphor-refantazio"
    ]
}



With the dictionary of nominees defined, the next step is to generate the corresponding **Metacritic URLs** for each game.  

- `base_url` → base link to Metacritic’s game section.  
- Looping through `goty_all`, we create the full URL for each game by combining the base path with the game’s identifier.  
- Each entry is stored in a list (`all_links`) as a dictionary containing:
  - **year** → the GOTY edition.  
  - **game** → the game identifier.  
  - **url** → the full Metacritic link.  

This structured list (`all_links`) serves as the foundation for the scraping process, ensuring we can systematically extract data for every nominee and winner across all years.


In [6]:
base_url="https://www.metacritic.com/game"

all_links = []

for year in goty_all:
    for game in goty_all[year]:
        full_link = f"{base_url}/{game}"
        all_links.append({
            "year": year,
            "game": game,
            "url": full_link
        })


*Output results:*

In [8]:
all_links

[{'year': 2014,
  'game': 'dragon-age-inquisition',
  'url': 'https://www.metacritic.com/game/dragon-age-inquisition'},
 {'year': 2014,
  'game': 'bayonetta-2',
  'url': 'https://www.metacritic.com/game/bayonetta-2'},
 {'year': 2014,
  'game': 'dark-souls-ii',
  'url': 'https://www.metacritic.com/game/dark-souls-ii'},
 {'year': 2014,
  'game': 'hearthstone-heroes-of-warcraft',
  'url': 'https://www.metacritic.com/game/hearthstone-heroes-of-warcraft'},
 {'year': 2014,
  'game': 'middle-earth-shadow-of-mordor',
  'url': 'https://www.metacritic.com/game/middle-earth-shadow-of-mordor'},
 {'year': 2015,
  'game': 'the-witcher-3-wild-hunt',
  'url': 'https://www.metacritic.com/game/the-witcher-3-wild-hunt'},
 {'year': 2015,
  'game': 'bloodborne',
  'url': 'https://www.metacritic.com/game/bloodborne'},
 {'year': 2015,
  'game': 'fallout-4',
  'url': 'https://www.metacritic.com/game/fallout-4'},
 {'year': 2015,
  'game': 'metal-gear-solid-v-the-phantom-pain',
  'url': 'https://www.metacritic.

### Extracting Game Titles

With the list of URLs ready, we use **Selenium** to open each Metacritic page and **BeautifulSoup** to parse the HTML content.  
The goal of this step is to validate the scraping pipeline by extracting the **game titles** directly from the page.  

Process:
1. Open each URL using Selenium (`driver.get()`).
2. Wait a few seconds (`time.sleep`) to ensure the page is fully loaded.
3. Parse the page with BeautifulSoup.
4. Locate the game title in the HTML (`c-productHero_title` class).
5. Save the result in the `titles` list, along with the corresponding year.  

This first extraction acts as a **sanity check**:  
- If the titles are correctly retrieved, it means the scraper can access and parse the pages as expected.  
- The ✅ success logs confirm that the setup is working before moving forward with deeper data collection (scores, reviews, genres, etc.).


In [16]:
driver=webdriver.Chrome()

titles=[]

for detail in all_links:
    driver.get(detail['url'])
    time.sleep(5)
    soup=BeautifulSoup(driver.page_source, 'html.parser')
    
    title_tag=soup.find('div', attrs={'class':'c-productHero_title g-inner-spacing-bottom-medium g-outer-spacing-top-medium'})
    if title_tag:
        title=title_tag.text.strip()
        titles.append({'Year': detail['year'],
                       'Title':title})
        print(f"✅ Sucess: {title}")
    
driver.quit()

✅ Sucess: Dragon Age: Inquisition
✅ Sucess: Bayonetta 2
✅ Sucess: Dark Souls II
✅ Sucess: Hearthstone: Heroes of Warcraft
✅ Sucess: Middle-earth: Shadow of Mordor
✅ Sucess: The Witcher 3: Wild Hunt
✅ Sucess: Bloodborne
✅ Sucess: Fallout 4
✅ Sucess: Metal Gear Solid V: The Phantom Pain
✅ Sucess: Super Mario Maker
✅ Sucess: Overwatch
✅ Sucess: DOOM
✅ Sucess: INSIDE
✅ Sucess: Titanfall 2
✅ Sucess: Uncharted 4: A Thief's End
✅ Sucess: The Legend of Zelda: Breath of the Wild
✅ Sucess: Horizon Zero Dawn
✅ Sucess: Persona 5
✅ Sucess: PlayerUnknown's Battlegrounds
✅ Sucess: Super Mario Odyssey
✅ Sucess: God of War
✅ Sucess: Assassin's Creed Odyssey
✅ Sucess: Celeste
✅ Sucess: Marvel's Spider-Man
✅ Sucess: Monster Hunter: World
✅ Sucess: Red Dead Redemption 2
✅ Sucess: Sekiro: Shadows Die Twice
✅ Sucess: Control
✅ Sucess: Death Stranding
✅ Sucess: Resident Evil 2
✅ Sucess: Super Smash Bros. Ultimate
✅ Sucess: The Outer Worlds
✅ Sucess: The Last of Us Part II
✅ Sucess: Animal Crossing: New Horiz

### Extracting Critic Scores

After confirming that the titles were being retrieved correctly, the next step was to collect the **Critic Score** for each game.  

Process:
1. Selenium loads each game page from the list of URLs.  
2. BeautifulSoup parses the HTML content.  
3. The script locates:
   - The **score value** (numerical rating given by critics).  
   - The **criteria information** (number of reviews used to calculate the score).  
4. Each result is stored in the `crit_scores` list as a dictionary, containing the critic score and its criteria.  
5. A success log (`✅ Success`) confirms that the data was extracted properly for each game.  

This step ensures that for every GOTY nominee and winner we capture not just the **critic rating**, but also the **review count**, which adds context to the reliability of the score.


In [13]:
driver=webdriver.Chrome()

crit_scores=[]

for detail in all_links:
    driver.get(detail['url'])
    time.sleep(5)
    soup=BeautifulSoup(driver.page_source, 'html.parser')
    
    cryt_tag=soup.find('div', attrs={'class':'c-productScoreInfo_text g-outer-spacing-right-auto'})
    href_tag=cryt_tag.find('a', attrs={'href':True})
    if href_tag:
        cryteria=href_tag.get_text().strip()
        
    main_tag=soup.find('div', attrs={'class':'c-siteReviewScore u-flexbox-column u-flexbox-alignCenter u-flexbox-justifyCenter g-text-bold c-siteReviewScore_green g-color-gray90 c-siteReviewScore_medium'})
    score_tag=main_tag.find('span')
    if score_tag:
        score=score_tag.text.strip()

    score_cryt= f'{score} ({cryteria})'
    crit_scores.append({'Critic_Score':score_cryt})
    print(f'✅ Success: {score_cryt} from {detail['game']}')

        
driver.quit()

✅ Success: 85 (Based on 45 Critic Reviews) from dragon-age-inquisition
✅ Success: 91 (Based on 80 Critic Reviews) from bayonetta-2
✅ Success: 91 (Based on 69 Critic Reviews) from dark-souls-ii
✅ Success: 88 (Based on 42 Critic Reviews) from hearthstone-heroes-of-warcraft
✅ Success: 84 (Based on 85 Critic Reviews) from middle-earth-shadow-of-mordor
✅ Success: 92 (Based on 79 Critic Reviews) from the-witcher-3-wild-hunt
✅ Success: 92 (Based on 100 Critic Reviews) from bloodborne
✅ Success: 87 (Based on 58 Critic Reviews) from fallout-4
✅ Success: 93 (Based on 86 Critic Reviews) from metal-gear-solid-v-the-phantom-pain
✅ Success: 88 (Based on 85 Critic Reviews) from super-mario-maker
✅ Success: 91 (Based on 64 Critic Reviews) from overwatch
✅ Success: 79 (Based on 66 Critic Reviews) from doom
✅ Success: 93 (Based on 87 Critic Reviews) from inside
✅ Success: 89 (Based on 61 Critic Reviews) from titanfall-2
✅ Success: 93 (Based on 113 Critic Reviews) from uncharted-4-a-thiefs-end
✅ Success:

### Extracting User Scores

In addition to critic reviews, Metacritic also aggregates **User Scores**, which reflect the community’s opinion of each game.  
This step collects both the **numerical rating** and the **number of user ratings** available for each title.  

Process:
1. For every game URL, Selenium loads the page and BeautifulSoup parses the HTML.  
2. The script searches for the `User Score` section specifically.  
3. If found:
   - The **score value** (from 0 to 10) is extracted.  
   - The **criteria** (number of user ratings) is also captured.  
4. Results are appended to the `user_scores` list and logged with a ✅ success message.  

Together with the Critic Score, the User Score provides a **balanced view** of each game’s reception, capturing both professional reviews and player feedback.


In [61]:
driver=webdriver.Chrome()

user_scores=[]

for detail in all_links:
    driver.get(detail['url'])
    time.sleep(5)
    soup=BeautifulSoup(driver.page_source, 'html.parser')
    
    main_tag=soup.find_all('div', attrs={'class':'c-productScoreInfo_text g-outer-spacing-right-auto'})
    for tag in main_tag:
        span_tag=tag.find('span', attrs={'class':'c-productScoreInfo_scoreTitle g-color-gray80 u-block u-text-uppercase g-text-variant-allPetiteCaps'})
        label=span_tag.text.strip()
        if label == 'User Score':
            href_tag=tag.find('a', attrs={'href':True})
            if href_tag:
                cryteria=href_tag.get_text().strip()
            
    main_tag=soup.find('div', attrs={'class':'c-siteReviewScore u-flexbox-column u-flexbox-alignCenter u-flexbox-justifyCenter g-text-bold c-siteReviewScore_green c-siteReviewScore_user g-color-gray90 c-siteReviewScore_medium'})
    score_tag=main_tag.find('span')
    if score_tag:
        score=score_tag.text.strip()

    score_cryt=f'{score} ({cryteria})'
    user_scores.append({'User_Score':score_cryt})
    print(f'✅ Success: {score_cryt} from {detail['game']}')
    
driver.quit()

✅ Success: 10 (Based on 5,450 User Ratings) from dragon-age-inquisition
✅ Success: 9.0 (Based on 1,517 User Ratings) from bayonetta-2
✅ Success: 7.7 (Based on 2,429 User Ratings) from dark-souls-ii
✅ Success: 10 (Based on 2,659 User Ratings) from hearthstone-heroes-of-warcraft
✅ Success: 7.9 (Based on 3,111 User Ratings) from middle-earth-shadow-of-mordor
✅ Success: 9.1 (Based on 17,812 User Ratings) from the-witcher-3-wild-hunt
✅ Success: 8.9 (Based on 13,701 User Ratings) from bloodborne
✅ Success: 10 (Based on 7,420 User Ratings) from fallout-4
✅ Success: 8.1 (Based on 6,829 User Ratings) from metal-gear-solid-v-the-phantom-pain
✅ Success: 8.6 (Based on 1,014 User Ratings) from super-mario-maker
✅ Success: 10 (Based on 7,434 User Ratings) from overwatch
✅ Success: 8.2 (Based on 1,290 User Ratings) from doom
✅ Success: 8.3 (Based on 1,651 User Ratings) from inside
✅ Success: 8.5 (Based on 2,959 User Ratings) from titanfall-2
✅ Success: 8.7 (Based on 16,640 User Ratings) from uncharte

### Extracting Release Dates

Another key piece of information is the **release date** of each game.  
This allows us to place every GOTY nominee and winner in a timeline, which can later be useful for identifying yearly trends in the industry.  

Process:
1. For each game URL, Selenium loads the page and BeautifulSoup parses the HTML.  
2. The script searches for the section containing the release date.  
3. If found, the date is extracted, cleaned, and stored in the `released_date` list.  
4. A ✅ success message confirms the value was captured.  

Collecting release dates provides important **contextual information**, helping us understand how GOTY nominees are distributed across years and how they align with industry cycles.


In [64]:
driver=webdriver.Chrome()

released_date=[]

for detail in all_links:
    driver.get(detail['url'])
    time.sleep(5)
    soup=BeautifulSoup(driver.page_source, 'html.parser')
    
    main_tag = soup.find_all('div', class_='g-text-xsmall')
    for tag in main_tag:
        date_tag=tag.find('span', class_='u-text-uppercase')
        if date_tag:
            date=date_tag.text.strip()
            released_date.append({'Released_Date':date})
            print(f'✅ Success: {date}')
            
driver.quit()

✅ Success: Nov 18, 2014
✅ Success: Oct 24, 2014
✅ Success: Mar 11, 2014
✅ Success: Mar 11, 2014
✅ Success: Sep 30, 2014
✅ Success: May 19, 2015
✅ Success: Mar 24, 2015
✅ Success: Nov 10, 2015
✅ Success: Sep 1, 2015
✅ Success: Sep 11, 2015
✅ Success: May 23, 2016
✅ Success: Nov 10, 2017
✅ Success: Jun 29, 2016
✅ Success: Oct 28, 2016
✅ Success: May 10, 2016
✅ Success: Mar 3, 2017
✅ Success: Feb 28, 2017
✅ Success: Apr 4, 2017
✅ Success: Dec 21, 2017
✅ Success: Oct 27, 2017
✅ Success: Apr 20, 2018
✅ Success: Oct 2, 2018
✅ Success: Jan 25, 2018
✅ Success: Sep 7, 2018
✅ Success: Jan 26, 2018
✅ Success: Oct 26, 2018
✅ Success: Mar 22, 2019
✅ Success: Aug 27, 2019
✅ Success: Nov 8, 2019
✅ Success: Jan 25, 2019
✅ Success: Dec 7, 2018
✅ Success: Oct 25, 2019
✅ Success: Jun 19, 2020
✅ Success: Mar 20, 2020
✅ Success: Mar 20, 2020
✅ Success: Apr 10, 2020
✅ Success: Jul 17, 2020
✅ Success: Sep 17, 2020
✅ Success: Mar 26, 2021
✅ Success: Sep 14, 2021
✅ Success: Oct 8, 2021
✅ Success: Aug 25, 2021


### Extracting Platforms

Another important attribute for each game is the list of **platforms** where it was released.  
This information is essential to understand the **market reach** of a title (exclusive releases vs. multi-platform launches).  

Process:
1. Selenium loads the page, and BeautifulSoup parses the HTML.  
2. The scraper locates the section containing platform details.  
3. Each platform is extracted individually (e.g., PlayStation, Xbox, PC, Nintendo).  
4. Results are stored as a list inside the `platforms` collection.  
5. A ✅ success message confirms that the platforms were appended successfully.  

By collecting platform data, we can later compare whether GOTY nominees are usually **exclusive to one console** or released across **multiple platforms**, which often influences both visibility and reception.


In [67]:
driver=webdriver.Chrome()

platforms=[]

for detail in all_links:
    driver.get(detail['url'])
    time.sleep(5)
    soup=BeautifulSoup(driver.page_source, 'html.parser')

    console_list=[]
    main_tag=soup.find('div', attrs={'class':'c-gameDetails_Platforms u-flexbox u-flexbox-row'})
    if main_tag:
        ul_tag=main_tag.find('ul', class_='g-outer-spacing-left-medium-fluid')
        if ul_tag:
            li_tag=ul_tag.find_all('li', attrs={'class':'c-gameDetails_listItem g-color-gray70 u-inline-block'})
            if li_tag:
                for li in li_tag:
                    console=li.text.strip()
                    console_list.append(console)
                    
    platforms.append({'Platform':console_list})
    print('Append Successful ✅')
            
driver.quit()


Append Successful ✅
Append Successful ✅
Append Successful ✅
Append Successful ✅
Append Successful ✅
Append Successful ✅
Append Successful ✅
Append Successful ✅
Append Successful ✅
Append Successful ✅
Append Successful ✅
Append Successful ✅
Append Successful ✅
Append Successful ✅
Append Successful ✅
Append Successful ✅
Append Successful ✅
Append Successful ✅
Append Successful ✅
Append Successful ✅
Append Successful ✅
Append Successful ✅
Append Successful ✅
Append Successful ✅
Append Successful ✅
Append Successful ✅
Append Successful ✅
Append Successful ✅
Append Successful ✅
Append Successful ✅
Append Successful ✅
Append Successful ✅
Append Successful ✅
Append Successful ✅
Append Successful ✅
Append Successful ✅
Append Successful ✅
Append Successful ✅
Append Successful ✅
Append Successful ✅
Append Successful ✅
Append Successful ✅
Append Successful ✅
Append Successful ✅
Append Successful ✅
Append Successful ✅
Append Successful ✅
Append Successful ✅
Append Successful ✅
Append Successful ✅


### Extracting Developers and Publishers

In this step, the scraper collects information about the **developers** and **publishers** of each game.  

- **Developers** → the studios responsible for creating the game.  
- **Publishers** → the companies responsible for distribution and marketing.  

Process:
1. Selenium loads each game page and BeautifulSoup parses the HTML.  
2. The script looks for both developer and publisher sections.  
3. If multiple entries exist (e.g., co-developers or co-publishers), all are captured and stored as lists.  
4. Each result is appended to the `devs` and `publishers` collections.  
5. ✅ Success messages confirm when developer and publisher information is found.  

This data is important to analyze **patterns of influence in the gaming industry** — for example, which studios consistently appear among GOTY nominees, or whether large publishers dominate the award landscape.


In [70]:
driver=webdriver.Chrome()

devs=[]
publishers=[]

for detail in all_links:
    driver.get(detail['url'])
    time.sleep(5)
    soup=BeautifulSoup(driver.page_source, 'html.parser')

    #Dev
    dev_list=[]
    dev_tag=soup.find('div', attrs={'class':'c-gameDetails_Developer u-flexbox u-flexbox-row'})
    if dev_tag:
        li_tag=dev_tag.find_all('li', class_='c-gameDetails_listItem u-inline-block g-color-gray70')
        if li_tag:
            for tag in li_tag:
                dev=tag.text.strip()
                dev_list.append(dev)
        else:
            a_tag=dev_tag.find('a', class_='u-text-underline')
            if a_tag:
                dev=a_tag.text.strip()
                dev_list.append(dev)
    devs.append({'Devs':dev_list})
    print(f'✅ Devs Found: {dev}')

    #Publisher
    pub_tag=soup.find('div', attrs={'class':'c-gameDetails_Distributor u-flexbox u-flexbox-row'})
    if pub_tag:
        span_tag=pub_tag.find('span', attrs={'class':'g-outer-spacing-left-medium-fluid u-block g-color-gray70'})
        if span_tag:
            publisher=span_tag.text.strip()
            publishers.append({'Publisher':publisher})
            print(f'✅ Publisher Found: {publisher}')
        else:
            href_tag=pub_tag.find('a', attrs={'class':'g-outer-spacing-left-medium-fluid u-block u-text-underline'})
            if href_tag:
                publisher=href_tag.text.strip()
                publishers.append({'Publisher':publisher})
                print(f'✅ Publisher Found: {publisher}')


driver.quit()

✅ Devs Found: BioWare
✅ Publisher Found: Electronic Arts
✅ Devs Found: PlatinumGames
✅ Publisher Found: Nintendo
✅ Devs Found: From Software
✅ Publisher Found: Namco Bandai Games
✅ Devs Found: Blizzard Entertainment
✅ Publisher Found: Blizzard Entertainment
✅ Devs Found: Monolith Productions
✅ Publisher Found: Warner Bros. Interactive Entertainment
✅ Devs Found: CD Projekt Red Studio
✅ Publisher Found: Warner Bros. Interactive Entertainment
✅ Devs Found: From Software
✅ Publisher Found: SCEA
✅ Devs Found: Bethesda Game Studios
✅ Publisher Found: Bethesda Softworks
✅ Devs Found: Moby Dick Studio
✅ Publisher Found: Konami
✅ Devs Found: Nintendo
✅ Publisher Found: Nintendo
✅ Devs Found: Blizzard Entertainment
✅ Publisher Found: Blizzard Entertainment
✅ Devs Found: Panic Button
✅ Publisher Found: Bethesda Softworks
✅ Devs Found: PLAYDEAD
✅ Publisher Found: PLAYDEAD
✅ Devs Found: Respawn Entertainment
✅ Publisher Found: Electronic Arts
✅ Devs Found: Naughty Dog
✅ Publisher Found: Sony Inter

### Extracting Genres

The next step focuses on collecting the **genre** of each game, which classifies the type of gameplay experience (e.g., RPG, Action, Platformer).  

Process:
1. Selenium loads the game’s Metacritic page and BeautifulSoup parses the content.  
2. The script locates the genre section and extracts its value.  
3. Each result is stored in the `genres` list.  
4. A ✅ success message confirms that the genre was successfully captured.  

Genres provide valuable context for analysis, as they allow us to explore patterns such as:  
- Which genres are most frequently nominated for GOTY.  
- Whether certain genres (e.g., RPGs or Action/Adventure games) have a higher chance of winning.  


In [72]:
driver=webdriver.Chrome()

genres=[]

for detail in all_links:
    driver.get(detail['url'])
    time.sleep(5)
    soup=BeautifulSoup(driver.page_source, 'html.parser')
    
    main_tag=soup.find('div', attrs={'class':'c-globalButton c-globalButton-small c-globalButton-primary'})
    genre=main_tag.text.strip()
    genres.append({'Genre':genre})
    print(f'✅ Sucess: {genre}')

driver.quit()


✅ Sucess: Western RPG
✅ Sucess: Linear Action Adventure
✅ Sucess: Action RPG
✅ Sucess: Card Battle
✅ Sucess: Open-World Action
✅ Sucess: Action RPG
✅ Sucess: Action RPG
✅ Sucess: Western RPG
✅ Sucess: Open-World Action
✅ Sucess: 2D Platformer
✅ Sucess: Tactical FPS
✅ Sucess: FPS
✅ Sucess: 2D Platformer
✅ Sucess: FPS
✅ Sucess: Linear Action Adventure
✅ Sucess: Open-World Action
✅ Sucess: Open-World Action
✅ Sucess: JRPG
✅ Sucess: Tactical FPS
✅ Sucess: 3D Platformer
✅ Sucess: Linear Action Adventure
✅ Sucess: Open-World Action
✅ Sucess: 2D Platformer
✅ Sucess: Open-World Action
✅ Sucess: Action RPG
✅ Sucess: Open-World Action
✅ Sucess: Action Adventure
✅ Sucess: Action Adventure
✅ Sucess: Open-World Action
✅ Sucess: Survival
✅ Sucess: 2D Fighting
✅ Sucess: Western RPG
✅ Sucess: Survival
✅ Sucess: Virtual Life
✅ Sucess: FPS
✅ Sucess: Action RPG
✅ Sucess: Open-World Action
✅ Sucess: Roguelike
✅ Sucess: Linear Action Adventure
✅ Sucess: FPS
✅ Sucess: Metroidvania
✅ Sucess: 3D Platformer
✅ 

### Extracting Review Links

Beyond scores and metadata, an essential component of this project is the **user reviews**.  
In this step, we build a list of links that point directly to the review pages for each game.  

Process:
1. For each game URL, Selenium loads the page and BeautifulSoup parses the HTML.  
2. The script searches for the **User Reviews** section.  
3. If found, the review link is extracted and combined with the base URL.  
4. Each valid review link is stored in the `review_links` list.  
5. A ✅ success log confirms when a review link has been successfully captured.  

Collecting these links is the foundation for the next stage, where the actual **user reviews** will be scraped and later analyzed (e.g., sentiment extraction).


In [10]:
driver = webdriver.Chrome()

base_url_review="https://www.metacritic.com"

review_links = []

for detail in all_links:
    driver.get(detail['url'])
    time.sleep(5)
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    main_tag = soup.find_all('div', attrs={'class': 'c-sectionHeader u-flexbox-row u-flexbox-alignMiddle'})
    
    user_review_found = False  

    for header in main_tag:
        if user_review_found:
            break
        
        label_tag = header.find('h2')
        if label_tag and label_tag.text.strip() == 'User Reviews':
            href_tag = header.find('a', attrs={'href': True})
            if href_tag:
                href = base_url_review + href_tag['href']
                review_links.append(href)
                print(f'✅ Success: {href}')
                user_review_found = True

driver.quit()

✅ Success: https://www.metacritic.com/game/dragon-age-inquisition/user-reviews/?platform=pc
✅ Success: https://www.metacritic.com/game/bayonetta-2/user-reviews/?platform=wii-u
✅ Success: https://www.metacritic.com/game/dark-souls-ii/user-reviews/?platform=playstation-3
✅ Success: https://www.metacritic.com/game/hearthstone-heroes-of-warcraft/user-reviews/?platform=pc
✅ Success: https://www.metacritic.com/game/middle-earth-shadow-of-mordor/user-reviews/?platform=playstation-4
✅ Success: https://www.metacritic.com/game/the-witcher-3-wild-hunt/user-reviews/?platform=playstation-4
✅ Success: https://www.metacritic.com/game/bloodborne/user-reviews/?platform=playstation-4
✅ Success: https://www.metacritic.com/game/fallout-4/user-reviews/?platform=playstation-4
✅ Success: https://www.metacritic.com/game/metal-gear-solid-v-the-phantom-pain/user-reviews/?platform=playstation-4
✅ Success: https://www.metacritic.com/game/super-mario-maker/user-reviews/?platform=wii-u
✅ Success: https://www.metacr

### Extracting the Quantity of User Reviews

Once the review links were collected, the next step was to extract the **total number of user reviews** available for each game.  

Process:
1. For each review link, Selenium loads the page and BeautifulSoup parses the HTML.  
2. The script identifies the block containing the **total count of user reviews**.  
3. The value is extracted, paired with the corresponding game title, and stored in the `all_reviews` list.  
4. A ✅ success message confirms when the review count has been successfully captured.  

This information is essential to contextualize the **User Score**, since games with a higher number of reviews generally provide a more reliable community evaluation than those with very few ratings.


In [24]:
all_reviews = []

driver = webdriver.Chrome()

for i, link in enumerate(review_links):
    driver.get(link)
    time.sleep(5)
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    block_tag = soup.find('div', class_='c-pageProductReviews_text')
    if block_tag:
        quantity=block_tag.text.strip()
        game_title = titles[i]['Title']
        all_reviews.append({'Title': game_title, 'Year_Quant': quantity})
        print(f'✅{quantity} for: {game_title}')

driver.quit()

✅Showing 1626 User Reviews for: Dragon Age: Inquisition
✅Showing 227 User Reviews for: Bayonetta 2
✅Showing 300 User Reviews for: Dark Souls II
✅Showing 638 User Reviews for: Hearthstone: Heroes of Warcraft
✅Showing 628 User Reviews for: Middle-earth: Shadow of Mordor
✅Showing 2610 User Reviews for: The Witcher 3: Wild Hunt
✅Showing 2966 User Reviews for: Bloodborne
✅Showing 1480 User Reviews for: Fallout 4
✅Showing 1183 User Reviews for: Metal Gear Solid V: The Phantom Pain
✅Showing 212 User Reviews for: Super Mario Maker
✅Showing 1550 User Reviews for: Overwatch
✅Showing 190 User Reviews for: DOOM
✅Showing 164 User Reviews for: INSIDE
✅Showing 558 User Reviews for: Titanfall 2
✅Showing 2980 User Reviews for: Uncharted 4: A Thief's End
✅Showing 6487 User Reviews for: The Legend of Zelda: Breath of the Wild
✅Showing 2935 User Reviews for: Horizon Zero Dawn
✅Showing 1171 User Reviews for: Persona 5
✅Showing 467 User Reviews for: PlayerUnknown's Battlegrounds
✅Showing 2537 User Reviews f

### Review Scraping — Baseline Loop (Initial Attempt)

This cell performs a first-pass scrape of **user review quotes**:
- Iterate over each review page URL.
- Parse the HTML and collect visible review blocks.
- Store each review text paired with its corresponding game title.

**Limitation:** this baseline loop only retrieved a **limited subset** of reviews.  
Due to how Metacritic loads content (pagination/infinite loading), only the reviews initially present in the page DOM were captured.  
We identified this constraint and, in the next steps, we address it with a more robust strategy (details intentionally omitted here).


In [ ]:
all_reviews = []

driver = webdriver.Chrome()

for i, link in enumerate(review_links):
    driver.get(link)
    time.sleep(5)
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    review_blocks = soup.find_all('div', attrs={'class': 'c-siteReview_quote g-outer-spacing-bottom-small'})
    
    for r, block in enumerate(review_blocks):
        span = block.find('span')
        if span:
            review_text = span.text.strip()
            game_title = titles[i]['Title']
            all_reviews.append({'Title': game_title, 'Review': review_text})
            print(f'✅ Review nº {r} for: {game_title} added')

driver.quit()

### Review Scraping Function (Year-Scoped + Infinite Scroll)

To overcome the limited capture of the baseline loop, we encapsulated the logic in
`scrape_reviews_by_title_and_year(...)`. This function reliably harvests **user reviews**
for a **specific year** while handling lazy loading/pagination.

**What it does**
- Receives the full `titles` and `review_links` lists plus a `target_year`.
- Opens each review page with Selenium and **repeatedly scrolls** to trigger dynamic loading.
- After each scroll, parses the DOM with BeautifulSoup and searches for **review blocks**
  across the main layout variants (multiple CSS selectors for robustness).
- Extracts **date** and **review text**; **keeps only reviews from `target_year`**.
- Uses a `captured_set` to **deduplicate** texts that reappear during loading.
- **Stops** when:
  - `max_scrolls` is reached, or
  - no new reviews are found for `max_consecutive_misses` consecutive iterations.
- Logs progress per scroll and returns a list


In [20]:
def scrape_reviews_by_title_and_year(titles, review_links, target_year, max_scrolls=600, max_consecutive_misses=200):
    from bs4 import BeautifulSoup
    import time, re
    from selenium import webdriver

    all_reviews = []
    captured_set = set()
    
    driver = webdriver.Chrome()

    for i, link in enumerate(review_links):
        game_title = titles[i]['Title']
        game_year = titles[i]['Year']

        if game_year != target_year:
            continue  

        print(f"\n🎯 Searching reviews for {game_title} ({game_year})")

        driver.get(link)
        time.sleep(3)

        scrolls = 0
        consecutive_misses = 0

        while scrolls < max_scrolls and consecutive_misses < max_consecutive_misses:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)

            soup = BeautifulSoup(driver.page_source, 'html.parser')
            
            review_blocks = soup.select(
                'div[data-testid="product-review"], div.c-siteReview_main, div.c-siteReview'
            )

            new_reviews = 0
            for block in review_blocks:
                date_tag  = block.select_one('div.c-siteReviewHeader_reviewDate, div.c-siteReview_reviewDate')
                quote_tag = block.select_one('div.c-siteReview_quote span')

                if not (date_tag and quote_tag):
                    continue

                date_text = date_tag.get_text(strip=True)
                m = re.search(r'(20\d{2})', date_text)
                if not m or int(m.group(1)) != target_year:
                    continue

                review_text = quote_tag.get_text(strip=True)
                if review_text and review_text not in captured_set:
                    all_reviews.append({
                        'Title': game_title,
                        'Review': review_text,
                        'Year': target_year
                    })
                    captured_set.add(review_text)
                    new_reviews += 1

            scrolls += 1
            if new_reviews == 0:
                consecutive_misses += 1
            else:
                consecutive_misses = 0

            print(f"🔁 Scroll {scrolls} — visíveis:{len(review_blocks)} — +{new_reviews} {target_year} — misses:{consecutive_misses}")

        print(f"✅ Reviews do ano-alvo esgotadas.")
        print(f"📦 Total reviews for {game_title}: {len([r for r in all_reviews if r['Title'] == game_title])}")

    driver.quit()
    return all_reviews





*Results:*

In [ ]:
all_reviews = []

for year in range(2014, 2025):  
    print(f"\n📅 Collecting reviews of {year}...")
    reviews = scrape_reviews_by_title_and_year(titles, review_links, year)
    all_reviews.extend(reviews)

*Example with 1 game only:*

In [ ]:
game = [{'Title': 'The Last of Us Part II', 'Year': 2020}]
review_game = ['https://www.metacritic.com/game/the-last-of-us-part-ii/user-reviews/?platform=playstation-4']

reviews = scrape_reviews_by_title_and_year(game, review_game, 2020)


### Deep Learning — Sentiment Classification of User Reviews

To transform raw user reviews into a structured signal, we apply **sentiment analysis** with a pretrained Transformer:

- **Model:** `cardiffnlp/twitter-roberta-base-sentiment` (RoBERTa fine-tuned for sentiment).
- **Output classes:** `Negative`, `Neutral`, `Positive`.
- **Why this model:** strong zero-shot performance on short, informal text; simple to integrate via 🤗 Transformers.

**Procedure**
1. Load tokenizer and model.
2. For each review:
   - Tokenize with truncation/padding (`max_length=512`).
   - Run a forward pass to obtain logits.
   - Apply `softmax` to convert logits to probabilities.
   - Select the label with highest probability and store it.
3. Save results alongside the original metadata: `Title`, `Year`, `Review`, `Sentiment`.

**Notes & caveats**
- The model is trained primarily on English social text; non-English or domain-specific phrasing (e.g., sarcasm, gaming slang) can reduce accuracy.
- Very long reviews are truncated to 512 tokens, so nuances at the end may be lost.
- This step is **descriptive** (not predictive): it enriches the dataset to support the final analysis and presentation.



In [77]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
import torch

MODEL = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)


### Sentiment Inference Loop

This cell applies the pretrained sentiment model to each **user review** and writes the
predicted label back into the record.

**Steps**
1. Iterate through all reviews with a progress bar (`tqdm`).
2. **Tokenize** each text with truncation and padding (`max_length=512`) to fit the model.
3. Run **inference** with `torch.no_grad()` to avoid gradient tracking.
4. Convert logits to probabilities with `softmax` and pick the label with the highest score
   among `['Negative', 'Neutral', 'Positive']`.
5. Store the result in a new field: `review['Sentiment'] = <label>`.

This transforms unstructured text into a usable categorical feature, enabling aggregate
views like per-game sentiment distribution and comparisons across years.


In [ ]:
from tqdm import tqdm
import numpy as np

for review in tqdm(reviews):
    text = review['Review']

    tokens = tokenizer(
        text,
        return_tensors='pt',
        truncation=True,
        max_length=512,
        padding=True
    )

    with torch.no_grad():
        output = model(**tokens)

    scores = softmax(output.logits.numpy()[0])
    labels = ['Negative', 'Neutral', 'Positive']
    sentiment = labels[np.argmax(scores)]

    review['Sentiment'] = sentiment


*Example of Results on Psychonauts 2:*

In [87]:
df = pd.DataFrame(reviews)

sentiment_summary = df.groupby(['Title', 'Sentiment']).size().unstack(fill_value=0).reset_index()


sentiment_summary.reset_index(drop=True, inplace=True)

print(sentiment_summary)


Sentiment          Title  Negative  Neutral  Positive
0          Psychonauts 2        83      148       400


### Merging All Scraped Attributes into a Single Dataset

After collecting each field separately, we consolidate everything into a **row-per-game** table.

**How it’s built**
- Each list (`titles`, `genres`, `devs`, `publishers`, `released_date`, `platforms`, `crit_scores`, `user_scores`) is **index-aligned**—the `i`-th element in every list refers to the same game.
- We merge the dictionaries for index `i` and append to `all_details`.
- Finally, we convert `all_details` into a DataFrame `df` and preview the result.

**Why this step matters**
- Produces a single, tidy schema with: `Year`, `Title`, `Genre`, `Devs`, `Publisher`, `Released_Date`, `Platform`, `Critic_Score`, `User_Score`.
- This becomes the **master table** used in the final website/presentation, while the reviews/sentiment aggregates are saved separately (e.g., `review_expan.csv/.xlsx`) for reference.



In [45]:
sentiment_summary.to_csv('review_expan.csv', index=False)

In [47]:
sentiment_summary.to_excel('review_expan.xlsx', index=False)

In [ ]:
all_details=[]
for i in range(len(titles)):
    merge={**titles[i], **genres[i], **devs[i], **publishers[i], **released_date[i], **platforms[i], **crit_scores[i], **user_scores[i]}
    all_details.append(merge)

df=pd.DataFrame(all_details)

In [105]:
df

,Year,Title,Genre,Devs,Publisher,Released_Date,Platform,Critic_Score,User_Score
0,2014,Dragon Age: Inquisition,Western RPG,[BioWare],Electronic Arts,"Nov 18, 2014","[Xbox One, PlayStation 3, Xbox 360, PC, PlaySt...",85 (Based on 45 Critic Reviews),"10 (Based on 5,450 User Ratings)"
1,2014,Bayonetta 2,Linear Action Adventure,[PlatinumGames],Nintendo,"Oct 24, 2014","[Wii U, Nintendo Switch]",91 (Based on 80 Critic Reviews),"9.0 (Based on 1,517 User Ratings)"
2,2014,Dark Souls II,Action RPG,[From Software],Namco Bandai Games,"Mar 11, 2014","[PlayStation 3, Xbox 360, PC]",91 (Based on 69 Critic Reviews),"7.7 (Based on 2,429 User Ratings)"
3,2014,Hearthstone: Heroes of Warcraft,Card Battle,[Blizzard Entertainment],Blizzard Entertainment,"Mar 11, 2014","[PC, iOS (iPhone/iPad)]",88 (Based on 42 Critic Reviews),"10 (Based on 2,659 User Ratings)"
4,2014,Middle-earth: Shadow of Mordor,Open-World Action,[Monolith Productions],Warner Bros. Interactive Entertainment,"Sep 30, 2014","[PlayStation 4, Xbox One, PlayStation 3, Xbox ...",84 (Based on 85 Critic Reviews),"7.9 (Based on 3,111 User Ratings)"
...,...,...,...,...,...,...,...,...,...
57,2024,Balatro,Card Battle,[LocalThunk],Playstack,"Feb 20, 2024","[PC, PlayStation 5, PlayStation 4, Nintendo Sw...",90 (Based on 32 Critic Reviews),"8.3 (Based on 1,264 User Ratings)"
58,2024,Black Myth: Wukong,Action RPG,[Game Science],Game Science,"Aug 19, 2024","[PC, PlayStation 5, Xbox Series X]",81 (Based on 87 Critic Reviews),"8.2 (Based on 7,667 User Ratings)"
59,2024,Elden Ring: Shadow of the Erdtree,Action RPG,[From Software],Bandai Namco Games,"Jun 21, 2024","[PC, Xbox Series X, PlayStation 5, PlayStation...",94 (Based on 64 Critic Reviews),"8.2 (Based on 3,948 User Ratings)"
60,2024,Final Fantasy VII Rebirth,Action RPG,[Square Enix],Square Enix,"Feb 29, 2024","[PlayStation 5, PC]",92 (Based on 145 Critic Reviews),"8.9 (Based on 6,065 User Ratings)"


### Joining Sentiment & Export

We now enrich the master table with the **sentiment summary** (per title).  
- **Join:** `LEFT` merge on `Title` (`df.merge(sentiment_summary, on='Title', how='left')`) to keep all games even if a title has no reviews.  
- **Result:** `df_final` containing the original metadata plus the sentiment columns (e.g., `Negative`, `Neutral`, `Positive`, and totals if present).  

Finally, we **persist** the deliverables for the website:
- `GOTY_YEARS.csv`  
- `GOTY_YEARS.xlsx`  



In [107]:
df_final = df.merge(sentiment_summary, on='Title', how='left')

In [111]:
df_final.to_csv('GOTY_YEARS.csv', index=False)

In [113]:
df_final.to_excel('GOTY_YEARS.xlsx', index=False)

### Final Cleaning & Column Layout

Before exporting, I applied a **basic string cleanup** to standardize fields:

- `.strip()` → trim leading/trailing whitespace.  
- `.replace()` → remove or normalize tokens (e.g., parenthetical notes like “(Based on … Reviews)”).  
- `.split()` → keep only the desired portion of a string when a field comes with extra text (e.g., keep the score before the first space/parenthesis).

> Note: the final table had **no missing values**, so no imputation was required.

I also reorganized the **column order** to improve readability by building a column list, removing a column, and inserting it at a specific position:

```python
cols = list(df.columns)
# Example: move 'column9' to come right after 'column3'
cols.remove('column9')
insert_at = df.columns.get_loc('column3') + 1
cols.insert(insert_at, 'column9')
df = df[cols]

## Conclusion

This notebook documents the pipeline used to extract, clean, and enrich GOTY data from Metacritic.  
It is intended as a technical reference rather than a presentation.  
All findings and visuals are presented on the website; ML was intentionally skipped due to limited data volume.